**QUANTUM KEY DISTRIBUTION** **SIMULATOR**

In [ ]:
!pip install qiskit

In [ ]:
!pip install qiskit_aer

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

def generate_random_bases(n):
    bases = np.random.randint(2, size=n)
    return bases

def encode_qubits(bits, bases):
    qc = QuantumCircuit(len(bits), len(bits))
    for i in range(len(bits)):
        if bits[i] == 1:
            qc.x(i)
        if bases[i] == 1:
            qc.h(i)
    return qc

def measure_qubits(qc, bases):
    for i in range(len(bases)):
        if bases[i] == 1:
            qc.h(i)
        qc.measure(i, i)
    return qc

def simulate_eavesdropping(alice_bits, alice_bases, bob_bases, eavesdrop_rate):
    n = len(alice_bits)
    num_eavesdropped = int(n * eavesdrop_rate)
    eavesdropped_bits = np.zeros(n, dtype=int)
    tampered_results = np.copy(alice_bits)

    eavesdrop_indices = np.random.choice(n, num_eavesdropped, replace=False)

    for i in eavesdrop_indices:
        eavesdropped_bits[i] = 1
        eve_basis = np.random.randint(2)
        if eve_basis == alice_bases[i]:
            eve_measurement = alice_bits[i]
        else:
            eve_measurement = np.random.randint(2)

        if bob_bases[i] != eve_basis:
            tampered_results[i] = alice_bits[i] if np.random.rand() > 0.75 else 1 - alice_bits[i]
        else:
            tampered_results[i] = eve_measurement if np.random.rand() > 0.5 else 1 - eve_measurement

    return eavesdropped_bits, tampered_results

def visualize_key_generation(alice_bits, alice_bases, bob_bases, bob_results,
                             sifted_key, eavesdropped_bits, error_rate):
    fig, axs = plt.subplots(6, 1, figsize=(10, 18))

    alice_1s = [bit if bit == 1 else 0 for bit in alice_bits]
    alice_0s = [0.8 if bit == 0 else 0 for bit in alice_bits]

    axs[0].bar(range(len(alice_1s)), alice_1s, color='blue', alpha=0.7,
               label="Bit = 1", width=0.8)
    axs[0].bar(range(len(alice_0s)), alice_0s, color='red', alpha=0.7,
               label="Bit = 0", width=0.6)
    axs[0].legend()
    axs[0].set_title("Alice's Bits")

    axs[1].bar(range(len(alice_bases)), alice_bases, color='green', alpha=0.7,
               label="Alice’s Bases", width=0.8)
    axs[1].bar(range(len(bob_bases)), np.array(bob_bases) * 0.8, color='orange', alpha=0.7,
               label="Bob’s Bases", width=0.6)
    axs[1].legend()
    axs[1].set_title("Bases (Alice vs. Bob)")
    axs[1].set_yticks([0, 1], labels=["Rectilinear", "Diagonal"])

    axs[2].bar(range(len(bob_results)), bob_results, color='purple', alpha=0.7,
               label="Bob’s Measurements", width=0.8)
    axs[2].legend()
    axs[2].set_title("Bob’s Results")

    axs[3].bar(range(len(eavesdropped_bits)), eavesdropped_bits, color='black',
               alpha=0.7, label="Eavesdropped Bits", width=0.8)
    axs[3].legend()
    axs[3].set_title("Eavesdropped Bits")

    table_data = []
    for i in range(len(alice_bits)):
        match_symbol = "✔" if alice_bases[i] == bob_bases[i] else "X"
        sifted_value = sifted_key[i]
        eve_symbol = "✔" if eavesdropped_bits[i] == 1 else "-"
        table_data.append([alice_bits[i], alice_bases[i], bob_bases[i],
                           match_symbol, bob_results[i], sifted_value, eve_symbol])

    col_labels = ["Alice's Bits", "Alice's Bases", "Bob's Bases", "Match?",
                  "Bob's Results", "Sifted Key", "Eavesdropped"]
    axs[4].axis('tight')
    axs[4].axis('off')
    axs[4].set_title("Key Generation Summary Table")
    table = axs[4].table(cellText=table_data, colLabels=col_labels, loc='center',
                         cellLoc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)

    axs[5].axis("off")
    message = f"Error Rate in Sifted Key: {error_rate:.2f}"
    if error_rate >= 0.25:
        message += "\nCommunication Compromised!!"
    axs[5].text(0.5, 0.5, message, fontsize=14, ha='center', va='center',
                color='red' if error_rate >= 0.25 else 'black')

    plt.tight_layout()
    plt.show()

# --- MAIN ---

num_bits = int(input("Enter number of bits: "))
eavesdrop_rate = float(input("Enter eavesdropping rate (0 to 1): "))

alice_bits = []
for i in range(num_bits):
    bit = int(input(f"Enter bit {i+1} for Alice (0 or 1): "))
    while bit not in [0, 1]:
        bit = int(input(f"Invalid input. Enter bit {i+1} for Alice (0 or 1): "))
    alice_bits.append(bit)

alice_bases = generate_random_bases(num_bits)
bob_bases = generate_random_bases(num_bits)

eavesdropped_bits, tampered_bits = simulate_eavesdropping(alice_bits, alice_bases, bob_bases,eavesdrop_rate)

qc = encode_qubits(tampered_bits, alice_bases)
qc = measure_qubits(qc, bob_bases)

simulator = AerSimulator()
compiled_circuit = transpile(qc, simulator)
result = simulator.run(compiled_circuit, shots=1, memory=True).result()

try:
    memory_data = result.get_memory()
    raw_bits = memory_data[0][::-1]
    bob_results = np.array([int(bit) for bit in raw_bits])
except KeyError:
    print("Error: No memory data available.")
    bob_results = np.zeros(num_bits)

sifted_key = []
for i in range(num_bits):
    if alice_bases[i] == bob_bases[i]:
        if eavesdropped_bits[i] == 1:
            sifted_key.append(int(1 - alice_bits[i]))
        else:
            sifted_key.append(int(alice_bits[i]))
    else:
        sifted_key.append("-")

matching_indices = [i for i in range(num_bits) if alice_bases[i] == bob_bases[i]]
errors = sum([1 for i in matching_indices if sifted_key[i] != alice_bits[i]])
error_rate = errors / len(matching_indices) if matching_indices else 0

visualize_key_generation(alice_bits, alice_bases, bob_bases, bob_results, sifted_key, eavesdropped_bits, error_rate)

print("Alice's Bits:    ", alice_bits)
print("Alice's Bases:   ", alice_bases)
print("Bob's Bases:     ", bob_bases)
print("Bob's Results:   ", bob_results)
print("Sifted Key:      ", sifted_key)
print(f"Error Rate in Sifted Key: {error_rate:.2f}")
